### Based on
https://learn.deeplearning.ai/courses/chatgpt-prompt-eng/lesson/2/guidelines

In [48]:
import openai
import os

print ("OpenAI version:", openai.__version__)
OPENAI_KEY = "sk-proj-yC8yLHxKJ2vknAYzEEAXT3BlbkFJ6csHffaax5hPfEm4lVsJ"
# openai.api_key  = os.getenv(OPENAI_KEY)

OpenAI version: 1.30.5


In [49]:
# client = openai.OpenAI(api_key=OPENAI_KEY)
client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Prompting Principles
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**

### Tactics

#### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
- Delimiters can be anything like: ```, """, < >, `<tag> </tag>`, `:`

In [55]:
text = f"""
You should express what you want a model to do by \ 
providing instructions that are as clear and \ 
specific as you can possibly make them. \ 
This will guide the model towards the desired output, \ 
and reduce the chances of receiving irrelevant \ 
or incorrect responses. Don't confuse writing a \ 
clear prompt with writing a short prompt. \ 
In many cases, longer prompts provide more clarity \ 
and context for the model, which can lead to \ 
more detailed and relevant outputs.
"""
prompt = f"""
Summarize the text delimited by triple backticks \ 
into a single sentence. Do not output anything else.
```{text}```
"""

response = get_completion(prompt, model="llama3")  # llama3, llama3:8b, llama3:instruct
print(response)

Provide clear and specific instructions to guide the model towards the desired output and reduce the chances of receiving irrelevant or incorrect responses.


#### Tactic 2: Ask for a structured output
- JSON, HTML

In [53]:
prompt = f"""
Generate a list of three made-up book titles along \ 
with their authors and genres. 
Provide them in JSON format with the following keys: 
book_id, title, author, genre.
Do not output anything else.
"""
response = get_completion(prompt, model="llama3")
print(response)

[
  {
    "book_id": 1,
    "title": "The Whispering Walls of Wysteria",
    "author": "Evelyn Everwood",
    "genre": "Fantasy"
  },
  {
    "book_id": 2,
    "title": "The Last Starlight Serenade",
    "author": "Caspian Nightshade",
    "genre": "Science Fiction"
  },
  {
    "book_id": 3,
    "title": "The Secret Garden of Shadows",
    "author": "Luna Moonwhisper",
    "genre": "Mystery"
  }
]


#### Tactic 3: Ask the model to check whether conditions are satisfied

In [58]:
text_1 = f"""
Making a cup of tea is easy! First, you need to get some \ 
water boiling. While that's happening, \ 
grab a cup and put a tea bag in it. Once the water is \ 
hot enough, just pour it over the tea bag. \ 
Let it sit for a bit so the tea can steep. After a \ 
few minutes, take out the tea bag. If you \ 
like, you can add some sugar or milk to taste. \ 
And that's it! You've got yourself a delicious \ 
cup of tea to enjoy.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"
Do not output anything else.

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt, model="llama3")
print("Completion for Text 1:")
print(response)

Completion for Text 1:
Step 1 - Get some water boiling.
Step 2 - Grab a cup and put a tea bag in it.
Step 3 - Once the water is hot enough, pour it over the tea bag.
Step 4 - Let it sit for a bit so the tea can steep.
Step 5 - After a few minutes, take out the tea bag.
Step 6 - If you like, add some sugar or milk to taste.
Step 7 - And that's it! You've got yourself a delicious cup of tea to enjoy.


In [60]:
text_2 = f"""
The sun is shining brightly today, and the birds are \
singing. It's a beautiful day to go for a \ 
walk in the park. The flowers are blooming, and the \ 
trees are swaying gently in the breeze. People \ 
are out and about, enjoying the lovely weather. \ 
Some are having picnics, while others are playing \ 
games or simply relaxing on the grass. It's a \ 
perfect day to spend time outdoors and appreciate the \ 
beauty of nature.
"""
prompt = f"""
You will be provided with text delimited by triple quotes. 
If it contains a sequence of instructions, \ 
re-write those instructions in the following format:

Step 1 - ...
Step 2 - …
…
Step N - …

If the text does not contain a sequence of instructions, \ 
then simply write \"No steps provided.\"
Do not output anything else.

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt, model="llama3")
print("Completion for Text 2:")
print(response)

Completion for Text 2:
No steps provided.


#### Tactic 4: "Few-shot" prompting

In [62]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest \ 
valley flows from a modest spring; the \ 
grandest symphony originates from a single note; \ 
the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt, model="llama3")
print(response)

<grandparent>: Ah, my young one, let us return to the river's source. You see, patience is not the absence of turmoil, but the gentle flow that shapes the landscape. It is the quiet persistence that wears down even the sturdiest boulders.

Just as the river's spring may be small and seemingly insignificant, our moments of patience can appear fleeting and inconsequential. Yet, it is in these quiet moments that we discover our greatest strength. For when we learn to flow with life's currents, rather than resisting their tides, we find ourselves carving out a path that is uniquely our own.

And now, my dear child, let us speak of resilience. Just as the river's valley is shaped by both its gentle and turbulent flows, so too do we face both calm and chaotic moments in our lives. But it is not the absence of turbulence that defines us; rather, it is how we respond to those challenges.

Resilience is not about being impervious to life's storms, but about learning to bend and flex like a will

### Principle 2: Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task

In [64]:
text = f"""
In a charming village, siblings Jack and Jill set out on \ 
a quest to fetch water from a hilltop \ 
well. As they climbed, singing joyfully, misfortune \ 
struck—Jack tripped on a stone and tumbled \ 
down the hill, with Jill following suit. \ 
Though slightly battered, the pair returned home to \ 
comforting embraces. Despite the mishap, \ 
their adventurous spirits remained undimmed, and they \ 
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple \
backticks with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Separate your answers with line breaks.
Do not output anything else.

Text:
```{text}```
"""
response = get_completion(prompt_1, model="llama3")
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Here are my answers:

1 - The text is about two siblings, Jack and Jill, who go on a quest to fetch water from a hilltop well but encounter misfortune along the way.

2 - La narration est sur deux frères, Jack et Jill, qui partent à la recherche d'eau dans un puits en haut de colline mais rencontrent malchance le long du chemin.

3 - The names mentioned in the French summary are:
- Jack
- Jill

4 - {"french_summary": "La narration est sur deux frères, Jack et Jill, qui partent à la recherche d'eau dans un puits en haut de colline mais rencontrent malchance le long du chemin.", "num_names": 2}


#### Ask for output in a specified format

In [72]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following \
keys: french_summary, num_names.

Do not output anything else.

Use the following format:
Summary: <summary>
Translation: <summary translation>
Names: <list of names in summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2, model="llama3")
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Here are the requested outputs:

Summary: Siblings Jack and Jill set out on a quest to fetch water from a hilltop well but encounter misfortune.

Translation: Les frères et sœurs Jack et Jill partent à la quête pour aller chercher de l'eau dans un puits en haut d'une colline mais rencontrent malchance.

Names: [Jack, Jill]

Output JSON:
{
"French_summary": "Les frères et sœurs Jack et Jill partent à la quête pour aller chercher de l'eau dans un puits en haut d'une colline mais rencontrent malchance",
"num_names": 2
}


#### Tactic 2: Instruct the model to work out its own solution before rushing to a conclusion

In [74]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt, model="llama3:instruct")
print(response)

The student's solution is correct!

Let's break it down:

1. Land cost: The land costs $100 per square foot, so the total land cost is $100x.
2. Solar panel cost: The solar panels cost $250 per square foot, so the total solar panel cost is $250x.
3. Maintenance cost: The maintenance contract has a flat fee of $100,000 and an additional $10 per square foot. So, the total maintenance cost is $100,000 + 10x.

The student correctly adds up these costs to get the total cost:

Total cost = Land cost + Solar panel cost + Maintenance cost
= 100x + 250x + (100,000 + 10x)
= 450x + 100,000

Well done, student!


#### Note that the student's solution is actually not correct.
#### We can fix this by instructing the model to work out its own solution first.
Note that it doesn't really end up working with llama3:8b below, perhaps llama3:70b would be able to solve this problem.

In [88]:
prompt = f"""
Your task is to determine the solution to the following problem \
and determine if the student's solution matches your solution.
Do not match with the student's solution until you've worked \
out your solution.

Use the following format:

Question:
```
I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet.
```

Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```

Your solution:
```
steps to work out the solution here
```

Let x be the size of the installation in square feet.
"""
response = get_completion(prompt, model="llama3")
print(response)

To determine the total cost for the first year of operations as a function of the number of square feet, I'll break down the costs and add them up.

1. Land cost: The land costs $100 per square foot, so the total land cost is 100x.
2. Solar panel cost: The solar panels cost $250 per square foot, so the total solar panel cost is 250x.
3. Maintenance cost: The maintenance contract has a flat fee of $100,000 and an additional $10 per square foot. This means the total maintenance cost is $100,000 + 10x.

To find the total cost, I'll add up these three costs:

Total cost = Land cost + Solar panel cost + Maintenance cost
= 100x + 250x + (100,000 + 10x)
= 350x + 100,000

So, the total cost for the first year of operations as a function of the number of square feet is 350x + 100,000.

Comparing my solution to the student's solution:

The student's solution is correct! They correctly identified the costs and added them up. The only difference is that they combined the land and solar panel costs